In [1]:
import pandas as pd # 데이터 전처리
import numpy as np # 데이터 전처리
import matplotlib.pyplot as plt # 데이터 시각화

from datetime import datetime, timedelta # 시간 데이터 처리
from statsmodels.tsa.arima_model import ARIMA # ARIMA 모델
%matplotlib inline

In [2]:
t_r48_med = pd.read_csv('D:/workspace/DACON_2019/SUHYUN/rolling/test/r48/med.csv')
submission = pd.read_csv('D:/workspace/DACON_2019/DACON_DATASET/submission.csv')

t_r48_med['Time'] = pd.to_datetime(t_r48_med['Time']) 
t_r48_med = t_r48_med.set_index('Time')

In [4]:
t_r48_med.head()

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017-07-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2750,0.021,NaN
2017-07-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2485,0.021,NaN
2017-07-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2370,0.021,NaN
2017-07-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2330,0.021,NaN
2017-07-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2290,0.021,NaN


In [5]:
submission.head()

,meter_id,X2018_7_1_1h,X2018_7_1_2h,X2018_7_1_3h,X2018_7_1_4h,X2018_7_1_5h,X2018_7_1_6h,X2018_7_1_7h,X2018_7_1_8h,X2018_7_1_9h,...,X2018_7_6_d,X2018_7_7_d,X2018_7_8_d,X2018_7_9_d,X2018_7_10_d,X2018_7_m,X2018_8_m,X2018_9_m,X2018_10_m,X2018_11_m
0,X102,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,X105,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,X106,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,X107,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,X110,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
place_id=[]; time=[] ; target=[] # 빈 리스트를 생성합니다.
for i in t_r48_med.columns:
    for j in range(len(t_r48_med)):
        place_id.append(i) # place_id에 미터 ID를 정리합니다.
        time.append(t_r48_med.index[j]) # time에 시간대를 정리합니다.
        target.append(t_r48_med[i].iloc[j]) # target에 전력량을 정리합니다.

new_df=pd.DataFrame({'place_id':place_id,'time':time,'target':target})
new_df=new_df.dropna() # 결측치를 제거합니다.
new_df=new_df.set_index('time') # time을 인덱스로 저장합니다.
new_df.head()

,place_id,target
time,,
2018-04-18 19:00:00,X26,2.5270
2018-04-18 20:00:00,X26,2.5840
2018-04-18 21:00:00,X26,2.5270
2018-04-18 22:00:00,X26,2.4475
2018-04-18 23:00:00,X26,2.3680


In [8]:
import itertools
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
pdq

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [9]:
def get_optimal_params(y):

    param_dict = {}
    for param in pdq:
        try:
            model = ARIMA(y, order=param)
            results_ARIMA = model.fit(disp=-1)
            param_dict[results_ARIMA.aic] = param
        except:
            continue

    min_aic = min(param_dict.keys())
    optimal_params = param_dict[min_aic]
    return optimal_params

In [10]:
agg={}
for key in new_df['place_id'].unique(): # 미터ID 200개의 리스트를 unique()함수를 통해 추출합니다.
    temp = new_df.loc[new_df['place_id']==key] # 미터ID 하나를 할당합니다.
    temp_1h=temp.resample('1h').sum() # 1시간 단위로 정리합니다.
    temp_1day=temp.resample('D').sum() # 1일 단위로 정리합니다.

    # 시간별 예측
    model = ARIMA(temp_1h['target'], order=get_optimal_params(temp_1h['target'])) # AIC를 최소화하는 최적의 파라미터로 모델링합니다.
    results_ARIMA = model.fit(disp=-1)
    fcst = results_ARIMA.forecast(24) # 24시간을 예측합니다.

    a = pd.DataFrame() # a라는 데이터프레임에 예측값을 정리합니다.
    
    for i in range(24):
        a['X2018_7_1_'+str(i+1)+'h']=[fcst[0][i]] # column명을 submission 형태에 맞게 지정합니다.

        
    # 일별 예측
    model = ARIMA(temp_1day['target'], order=get_optimal_params(temp_1day['target'])) # AIC를 최소화하는 최적의 파라미터로 모델링합니다.
    results_ARIMA = model.fit(disp=-1)
    fcst = results_ARIMA.forecast(10) # 10일을 예측합니다.

    for i in range(10):
        a['X2018_7_'+str(i+1)+'_d']=[fcst[0][i]] # column명을 submission 형태에 맞게 지정합니다.
    
    
    # 월별 예측
    # 일별로 예측하여 7월 ~ 11월의 일 수에 맞게 나누어 합산합니다.
    fcst = results_ARIMA.forecast(153)
    a['X2018_7_m'] = [np.sum(fcst[0][:31])] # 7월 
    a['X2018_8_m'] = [np.sum(fcst[0][31:62])] # 8월
    a['X2018_9_m'] = [np.sum(fcst[0][62:92])] # 9월
    a['X2018_10_m'] = [np.sum(fcst[0][92:123])] # 10월
    a['X2018_11_m'] = [np.sum(fcst[0][123:153])] # 11월
    
    a['meter_id'] = key 
    agg[key] = a[submission.columns.tolist()]
    print(key)
print('---- Modeling Done ----')

C:\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


X26
X303
X241
X435


C:\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


X402
X352


C:\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


...

X283
X329
X223
X266
X20
X443
X347
X75


C:\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


X107


C:\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


X230
---- Modeling Done ----


In [11]:
output1 = pd.concat(agg, ignore_index=False)
output2 = output1.reset_index().drop(['level_0','level_1'], axis=1)
output2['id'] = output2['meter_id'].str.replace('X','').astype(int)
output2 =  output2.sort_values(by='id', ascending=True).drop(['id'], axis=1).reset_index(drop=True)
output2.to_csv('D:/workspace/DACON_2019/SUHYUN/sub_baseline_r48_med.csv', index=False)

In [12]:
output2.head()

,meter_id,X2018_7_1_1h,X2018_7_1_2h,X2018_7_1_3h,X2018_7_1_4h,X2018_7_1_5h,X2018_7_1_6h,X2018_7_1_7h,X2018_7_1_8h,X2018_7_1_9h,...,X2018_7_6_d,X2018_7_7_d,X2018_7_8_d,X2018_7_9_d,X2018_7_10_d,X2018_7_m,X2018_8_m,X2018_9_m,X2018_10_m,X2018_11_m
0,X5,6.414162,6.415860,6.416758,6.417495,6.418199,6.418896,6.419592,6.420288,6.420984,...,161.044958,162.751451,164.457944,166.164437,167.870930,5521.406509,7161.346245,8491.776136,10388.324435,11614.658255
1,X7,1.485958,1.484842,1.484238,1.483917,1.483752,1.483673,1.483642,1.483637,1.483647,...,29.247674,29.144667,29.092829,29.066741,29.053612,913.181432,900.249655,871.209344,900.249655,871.209344
2,X10,3.226025,3.226370,3.227080,3.227890,3.228728,3.229574,3.230422,3.231271,3.232120,...,58.575933,58.575933,58.575933,58.575933,58.575933,1823.967307,1815.853908,1757.277976,1815.853908,1757.277976
3,X16,5.026338,5.027509,5.028437,5.029315,5.030182,5.031048,5.031912,5.032777,5.033642,...,84.751891,84.563237,84.483593,84.449970,84.435775,2659.320752,2617.187513,2532.762110,2617.187513,2532.762110
4,X18,2.578535,2.578574,2.578614,2.578656,2.578700,2.578744,2.578789,2.578835,2.578881,...,65.035048,65.693973,66.352897,67.011822,67.670747,2220.353130,2853.579726,3364.444829,4099.606253,4570.276952


In [115]:
from SMAPE import smape_fast

In [118]:
from numba import jit
import math

In [119]:
test_ARIMA=pd.read_csv("D:/workspace/DACON_2019/SUHYUN/output/test/ARIMA.csv")
test_soomin = pd.read_csv('D:/workspace/DACON_2019/SUHYUN/output/test/avg_per_hour_ARIMA.csv')

In [122]:
y_true=test_ARIMA.drop(test_ARIMA.columns[0], axis=1).values.reshape(-1)

In [123]:
y_pred=test_soomin.drop(test_soomin.columns[0], axis=1).values.reshape(-1)

In [135]:
out = 0

for i in range(y_true.shape[0]):
    a = y_true[i]
    b = y_pred[i]
    c = a+b
    if c == 0:
        continue
    out += math.fabs(a - b) / c
out *= (200.0 / y_true.shape[0])
print(out)

9.679389994147028


In [117]:
smape_fast(test_ARIMA, test_soomin)

KeyError: 0

In [112]:
test_ARIMA.drop(test_ARIMA.columns[0], axis=1).values.reshape(-1)

array([  6.79933654,   6.60844378,   6.50222579, ..., 175.7429663 ,
       181.6010652 , 175.7429663 ])

In [105]:
a1 = test_ARIMA1.values.reshape(-1)

In [19]:
def smape_fast(df_y_true, df_y_pred):
    from numba import jit
    import math
    out = 0
    y_true = df_y_true.drop(df_y_true.columns[0], axis=1).values.reshape(-1)
    y_pred = df_y_pred.drop(df_y_pred.columns[0], axis=1).values.reshape(-1)    
    for i in range(y_true.shape[0]):
        a = y_true[i]
        b = y_pred[i]
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

In [106]:
a1

array([  6.79933654,   6.60844378,   6.50222579, ..., 175.7429663 ,
       181.6010652 , 175.7429663 ])

In [107]:
test_soomin1 = test_soomin.drop(test_soomin.columns[0], axis=1)
b1 = test_soomin1.values.reshape(-1)

In [109]:
b1[1]

6.6624214798838

In [110]:
smape_fast(a1, b1)

9.679389994147028

In [86]:
c = 1

In [88]:
if c ==  0:
    print('true')
else:
    print('False')

False


In [41]:
a = np.arange(1,10)
a
b = np.arange(5,14)

In [42]:
b

9

In [20]:
smape_fast(test_ARIMA, output2)

KeyError: 0